# Project IRIS: INFOBRAS Data Cleaning and Certification

**Lead Data Engineer:** Jhon Wilber Ajata Ascarrunz

**Objective:** This notebook ingests the raw INFOBRAS public works dataset (`DataSet-Obras-Publicas-23-07-2025.xlsx`), performs a full cleaning, auditing, and curation pipeline, and exports a certified, analysis-ready data asset (`infobras_certified_v1.csv`). This process forms the foundational data layer for Project IRIS.

In [1]:
# --- Step 1: Environment Setup & Data Ingestion ---
import pandas as pd
import numpy as np

print("Libraries imported successfully.")

# Define the path to the raw Excel file
excel_path = '../data/DataSet-Obras-Publicas-23-07-2025.xlsx'

try:
    # Load the Excel file using the openpyxl engine
    df_raw = pd.read_excel(excel_path, engine='openpyxl')
    print(f"Successfully loaded raw data from '{excel_path}'.")
    print(f"Initial shape: {df_raw.shape[0]} rows, {df_raw.shape[1]} columns.")
except FileNotFoundError:
    print(f"CRITICAL ERROR: The file was not found at the specified path: {excel_path}")
    print("Please ensure the raw data file exists before proceeding.")

Libraries imported successfully.
Successfully loaded raw data from '../data/DataSet-Obras-Publicas-23-07-2025.xlsx'.
Initial shape: 180941 rows, 113 columns.


In [2]:
# --- Step 2: The Data Processing Pipeline Function ---

def clean_infobras_data(raw_df):
    """
    This function takes the raw INFOBRAS DataFrame and executes a complete
    cleaning, auditing, and curation pipeline. (Version 3.0)
    
    Args:
        raw_df (pd.DataFrame): The raw DataFrame loaded from the Excel file.
    
    Returns:
        pd.DataFrame: A certified, analysis-ready DataFrame.
    """
    df = raw_df.copy()
    print("Cleaning pipeline initiated...")

    # --- Step 2.1: Shielding Text Data ---
    print("  -> Step 2.1: Shielding text columns (stripping whitespace)...")
    text_cols = df.select_dtypes(include=['object']).columns
    for col in text_cols:
        df[col] = df[col].str.strip()
    
    # --- Step 2.2: Standardizing Column Names ---
    print("  -> Step 2.2: Standardizing column names to snake_case...")
    df.columns = (df.columns.str.lower()
                  .str.replace(' ', '_', regex=False).str.replace('¿', '', regex=False)
                  .str.replace('?', '', regex=False).str.replace('(', '', regex=False)
                  .str.replace(')', '', regex=False).str.replace(':', '', regex=False)
                  .str.replace('.', '', regex=False).str.normalize('NFKD')
                  .str.encode('ascii', errors='ignore').str.decode('utf-8'))

    # --- Step 2.3: Repairing Structural Defects ---
    print("  -> Step 2.3: Repairing structure (renaming duplicated columns)...")
    rename_dict = {
        'ruc1': 'supervisor_ruc', 'nombre_o_razon_social_de_la_empresa_o_consorcio1': 'supervisor_name',
        'monto_del_contrato__en_soles1': 'supervisor_contract_amount_soles', 'tipo_de_documento_de_identidad1': 'resident_id_type',
        'numero_de_documento1': 'resident_id_number', 'nombres_apellidos1': 'resident_full_name',
        'colegiatura1': 'resident_professional_org', 'numero_de_colegiatura1': 'resident_professional_id',
        'fecha_inicio_de_labores': 'resident_start_date', 'fecha_fin__de_labores': 'resident_end_date'
    }
    df = df.rename(columns=rename_dict)

    # --- Step 2.4: Pruning Uninformative Columns ---
    print("  -> Step 2.4: Pruning uninformative (mostly empty) columns...")
    cols_to_prune = ['fecha_de_aprobacion', 'otra_marca', 'tipo_de_certificado_de_inversion_publica', 
                     'numero_del_cipril_/cipgn', 'fecha_del_cipril_/_cipgn', 'monto_cipril_/cipgn']
    df = df.drop(columns=cols_to_prune, errors='ignore')
    
    # --- Step 2.5: Transforming Data Types (Dates) ---
    print("  -> Step 2.5: Transforming date columns...")
    date_cols = ['fecha_de_actualizacion', 'fecha_de_aprobacion_del_expediente', 'fecha_inicio_supervision', 'fecha_fin_supervision', 
                 'resident_start_date', 'resident_end_date', 'fecha_de_inicio_de_obra', 'fecha_finalizacion_programada_de_obra', 
                 'fecha_de_entrega_del_terreno', 'fecha_de_registro_de_avance', 'fecha_de_paralizacion', 
                 'fecha_finalizacion_reprogramada_de_obra', 'fecha_de_finalizacion_real', 'fecha_de_recepcion', 
                 'fecha_de_aprobacion_de_liquidacion_de_obra', 'fecha_de_transferencia']
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')
            
    # --- Step 2.6: Integrated Auditing & Correction ---
    print("  -> Step 2.6: Auditing and correcting for temporal coherence...")
    inconsistent_mask = (df['fecha_de_finalizacion_real'].notna()) & (df['fecha_de_inicio_de_obra'].notna()) & (df['fecha_de_finalizacion_real'] < df['fecha_de_inicio_de_obra'])
    num_inconsistent = inconsistent_mask.sum()
    if num_inconsistent > 0:
        df = df[~inconsistent_mask].copy()
        print(f"     - Removed {num_inconsistent} rows with incoherent dates.")
    
    # --- Step 2.7: Transforming Data Types (Numerics) ---
    print("  -> Step 2.7: Decontaminating and transforming numeric columns...")
    contaminated_numeric_cols = ['monto_viable/aprobado', 'monto_de_aprobacion_de_expediente_tecnico', 'tasa_de_cambio', 'monto_aprobado_en_soles', 'monto_del_contrato__en_soles', 'supervisor_contract_amount_soles', 'porcentaje_de_terreno_entregado', 'avance_fisico_programado_acumulado_%', 'avance_fisico_real_acumulado_%', 'monto_de_valorizacion_programado_acumulado', 'monto_de_valorizacion_ejecutado_acumulado', 'porcentaje_de_ejecucion_financiera', 'monto_de_ejecucion_financiera_de_la_obra', 'monto_de_adicionales_de_obra_en_soles', 'monto_de_adicionales_de_supervision_en_soles', 'monto_de_deductivos_de_obra_en_soles', 'costo_de_la_obra_en_soles', 'monto_total_devengado_del_proyecto']
    for col in contaminated_numeric_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace('S/.', '', regex=False).str.replace(',', '', regex=False).str.replace('%', '', regex=False).str.strip()
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # --- Step 2.8: Strategic Null Value Imputation ---
    print("  -> Step 2.8: Handling null values strategically...")
    df['causal_de_paralizacion'] = df['causal_de_paralizacion'].fillna('Not Paralyzed')
    df['corresponde_a_un_saldo_de_obra'] = df['corresponde_a_un_saldo_de_obra'].fillna('No')
    key_categorical_cols = ['departamento', 'provincia', 'distrito', 'modalidad_de_ejecucion_de_la_obra', 'estado_de_ejecucion']
    for col in key_categorical_cols:
        if col in df.columns:
            df[col] = df[col].fillna('Unknown')
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    df[numeric_cols] = df[numeric_cols].fillna(0)
    object_cols = df.select_dtypes(include=['object']).columns.tolist()
    for col in object_cols:
        df[col] = df[col].fillna('Not Applicable')

    # --- Step 2.9: Data Curation (Plausibility Filters) ---
    print("  -> Step 2.9: Applying business plausibility filters (curation)...")
    initial_rows = len(df)
    df = df[df['plazo_de_ejecucion_en_dias'] >= 1]
    print(f"     - Removed {initial_rows - len(df)} rows with non-positive timelines.")
    initial_rows = len(df)
    df = df[df['plazo_de_ejecucion_en_dias'] <= 3650] # Capping at 10 years
    print(f"     - Removed {initial_rows - len(df)} rows with extreme timelines (> 10 years).")

    print("\nPIPELINE COMPLETED: Cleaning, auditing, and curation finished!")
    return df

print("Data processing function `clean_infobras_data` defined successfully.")

Data processing function `clean_infobras_data` defined successfully.


In [3]:
# --- Step 3: Execute Pipeline and Export Certified Asset ---

# Execute the complete cleaning, auditing, and curation pipeline
df_certified = clean_infobras_data(df_raw)

# Save the final, certified result to a new CSV file
certified_path = '../data/infobras_certified_v1.csv'
df_certified.to_csv(certified_path, index=False)

print("\n----------------------------------------------------")
print("✅  PROCESS FINISHED SUCCESSFULLY")
print(f"File '{certified_path}' has been exported.")
print("This file is clean, audited, curated, and ready for analysis.")
print(f"Final shape: {df_certified.shape[0]} rows, {df_certified.shape[1]} columns.")

Cleaning pipeline initiated...
  -> Step 2.1: Shielding text columns (stripping whitespace)...
  -> Step 2.2: Standardizing column names to snake_case...
  -> Step 2.3: Repairing structure (renaming duplicated columns)...
  -> Step 2.4: Pruning uninformative (mostly empty) columns...
  -> Step 2.5: Transforming date columns...


C:\Users\GAMERS\AppData\Local\Temp\ipykernel_13324\4046884841.py:58: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')


  -> Step 2.6: Auditing and correcting for temporal coherence...
     - Removed 898 rows with incoherent dates.
  -> Step 2.7: Decontaminating and transforming numeric columns...
  -> Step 2.8: Handling null values strategically...
  -> Step 2.9: Applying business plausibility filters (curation)...
     - Removed 47887 rows with non-positive timelines.
     - Removed 19 rows with extreme timelines (> 10 years).

PIPELINE COMPLETED: Cleaning, auditing, and curation finished!

----------------------------------------------------
✅  PROCESS FINISHED SUCCESSFULLY
File '../data/infobras_certified_v1.csv' has been exported.
This file is clean, audited, curated, and ready for analysis.
Final shape: 132137 rows, 108 columns.
